<a href="https://colab.research.google.com/github/DataTak/text_data_analysis/blob/main/excercise/w12_text_data_analysis_tfidf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 16.2 MB/s eta 0:00:00


<br>
<br>

# Bag-of-Words

In [4]:
from konlpy.tag import Okt

#OKT 인스턴스 생성
tokenizer = Okt()

#분석할 문장
text = "나의 살던 고향은 꽃피는 산골"

#형태소 분석(형태소 기준으로 쪼개어라!)
tokens = tokenizer.morphs(text)

print(tokens)

['나', '의', '살던', '고향', '은', '꽃', '피는', '산골']


In [5]:
# 단어-인덱스 딕셔너리(키와 벨류로 정해졌음.)
word_to_index = {}

# 토큰을 인덱스로 변환
for token in tokens:
    if token not in word_to_index.keys():
        word_to_index[token] = len(word_to_index)

print(word_to_index)

{'나': 0, '의': 1, '살던': 2, '고향': 3, '은': 4, '꽃': 5, '피는': 6, '산골': 7}


In [6]:
# BoW로 변환
def convert_bow(sentence, word_to_index):

    # 벡터를 단어의 개수만큼 0으로 초기화
    vector = [0]*(len(word_to_index))

    # 문장을 토큰으로 분리
    tokenizer = Okt()
    tokens = tokenizer.morphs(sentence)

    # 단어의 인덱스 위치에 1 설정
    for token in tokens:
        if token in word_to_index.keys():
            vector[word_to_index[token]] += 1

    return vector

In [7]:
print(word_to_index)

{'나': 0, '의': 1, '살던': 2, '고향': 3, '은': 4, '꽃': 5, '피는': 6, '산골': 7}


In [8]:
convert_bow("복숭아 꽃 살구 꽃 아기 진달래", word_to_index)

[0, 0, 0, 0, 0, 2, 0, 0]

In [9]:
convert_bow("나의 고향은 산골입니다.", word_to_index)

[1, 1, 0, 1, 1, 0, 0, 1]

<br>
<br>

# TF-IDF

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

documents = [
    '나의 살던 고향은 꽃 피는 산골',
    '복숭아 꽃 살구 꽃 아기 진달래',
    '울긋불긋 꽃 대궐 차리인 동네',
    '그 속에서 놀던 때가 그립습니다'
]

# TF-IDF로 벡터화
# 1글자도 인식이 되도록 토큰 패턴 변경
tf_idf = TfidfVectorizer(token_pattern = r"(?u)\b\w+\b")
tf_idf.fit(documents)

print(tf_idf.vocabulary_)

{'나의': 4, '살던': 12, '고향은': 0, '꽃': 3, '피는': 18, '산골': 10, '복숭아': 9, '살구': 11, '아기': 14, '진달래': 16, '울긋불긋': 15, '대궐': 6, '차리인': 17, '동네': 7, '그': 1, '속에서': 13, '놀던': 5, '때가': 8, '그립습니다': 2}


In [11]:
tf_idf_dict = {v:k for k,v in tf_idf.vocabulary_.items()}

In [12]:
# 다른 문서에도 많이 나온 단어는 낮은 수치
tf_idf.transform(['꽃 동네 새 동네 나의 옛고향']).toarray()

array([[0.        , 0.        , 0.        , 0.27448674, 0.43003652,
        0.        , 0.        , 0.86007303, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ]])

In [13]:
tf_idf_dict[3], tf_idf_dict[4], tf_idf_dict[7]

('꽃', '나의', '동네')

In [14]:
# 같은 문서에 많이 나온 단어는 높은 수치
tf_idf.transform(['복숭아 같은 아기 얼굴, 진달래 같은 아기 얼굴']).toarray()

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.40824829,
        0.        , 0.        , 0.        , 0.        , 0.81649658,
        0.        , 0.40824829, 0.        , 0.        ]])

In [15]:
tf_idf_dict[9], tf_idf_dict[14], tf_idf_dict[16]

('복숭아', '아기', '진달래')